## Start compute simliarity between two strings

In [1]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))
sys.path.append(project_root)
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [2]:
from src.dataset.coupert import CoupertDataset
from src.arguments import DataArguments
from utils.retrieval import retrieval
from utils.metrics import compute_recall
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
from safetensors.torch import save_file, load_file

model_dir = "../model/Marqo/marqo-ecommerce-embeddings-L"
embedding_path = os.path.join(
    "../embeddings/", model_dir.split("/")[-1] + ".safetensors"
)
data_dir = "../data/coupert"
device = "cuda:1"


/home/hutu/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hutu/miniconda3/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.0 (you have 1.4.24). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
embs = load_file(embedding_path)
eval_embs = embs["eval_embs"]
eval_ids = embs["eval_ids"]
gallery_embs = embs["gallery_embs"]
gallery_ids = embs["gallery_ids"]

# 按id将embs排序
eval_embs_pos = torch.argsort(eval_ids)
gallery_embs_pos = torch.argsort(gallery_ids)
eval_embs = eval_embs[eval_embs_pos]
gallery_embs = gallery_embs[gallery_embs_pos]

In [4]:
eval_df = pd.read_json("../data/coupert/coupert_eval.jsonl", lines=True)
gallery_df = pd.read_json("../data/coupert/coupert_gallery.jsonl", lines=True)


In [5]:
gallery_ids = gallery_df["dp-id"].values.tolist()
len(gallery_ids), gallery_embs.shape

(2488144, torch.Size([2488144, 2048]))

In [6]:
ret_ids = retrieval(
    eval_embs, gallery_embs, gallery_ids=gallery_ids, topK=50, device=device
)
eval_df["pred"] = ret_ids

Chunk Size: 512, 32 chunks
Gallery Chunk Size: 512, 4860 chunks


100%|██████████| 32/32 [07:55<00:00, 14.86s/it]


In [7]:
eval_df["recall"] = eval_df.apply(compute_recall("pred"), axis=1)
eval_df.recall.mean()

0.8415787924926105

In [8]:
eval_df.to_json(
    "../data/coupert/coupert_eval_retrieval.jsonl", lines=True, orient="records"
)

In [9]:
# 找出recall<0.5的, 查看label的title和预测的title
df = eval_df[eval_df.recall == 0]
for i, row in df.iterrows():
    print("=" * 80)
    print(f"id:{row['dp-id']}")
    print(row["title"])
    print(row["pred"])
    print(row["recall"])
    print(row["label"])
    label = gallery_df[gallery_df["dp-id"].isin(row["label"])]["title"].values
    pred = gallery_df[gallery_df["dp-id"].isin(row["pred"])]["title"].values
    print(label)
    print(pred)

    if i > 10:
        break

id:100880478-276
General Altimax 365AW All Weather 235/65R18 106H Passenger Tire
['63756558-0', '3285357502-0', '97342521-0', '103744749-0', '859303089-0', '263856407-0', '3728682553-0', '1944496419-0', '1802075194-0', '1941630913-0', '641605974-0', '1025875951-0', '2143074354-0', '2485909690-0', '2063507715-0', '3046923580-0', '1692728746-0', '2949535540-0', '1243000525-0', '973816073-0', '2112996849-0', '3703146430-0', '2783511058-0', '1049779333-0', '988344183-0', '1519383800-0', '4241923034-0', '2115352896-0', '3486233382-0', '4090830469-0', '2257690257-0', '841597656-0', '2117645699-0', '771013140-0', '1069520048-0', '3110130633-0', '2956862689-0', '834186315-0', '426765928-0', '1025092305-0', '3193998679-0', '3470158941-0', '2175762130-0', '3328807914-0', '2194726494-0', '877041296-0', '408978891-0', '2500948563-0', '1090655876-0', '3370662454-0']
0.0
['1770048153-0', '1782655961-0', '4250988828-0', '592391230-0', '4111365012-0', '3437314108-0', '3807079623-0', '2044313669-0', '1